# Urban Heat Island Analysis – Bologna

This notebook performs a comprehensive geospatial analysis to identify and classify Urban Heat Island (UHI) effects in the city of Bologna, Italy, using satellite data and derived indices.

## Process Overview

###  Index Summary

All files are saved in both `.tif` and `.geojson` formats, clipped to the Bologna boundary.

| Index | Description | File name (no extension) |
|-------|-------------|----------------------------|
| **LST** | Land Surface Temperature derived from Landsat ST_B10 | lst_mean |
| **Z-score of LST** | Standardized anomaly of LST to detect relative hot/cold zones | z_score |
| **NDVI** | Normalized Difference Vegetation Index from bands B4/B5 | ndvi_mean |
| **Albedo** | Surface reflectivity index (Liang, 2001) using bands B2-B7 | albedo_mean |
| **Heat-Vegetation Index** | LST_norm – NDVI_norm, shows lack of vegetative cooling | heat_veg_index |
| **Heat Retention Index** | LST_norm – Albedo_norm, shows capacity to retain heat | heat_retention_index |
| **MODIS ΔLST** | Day-Night temperature difference from MODIS | delta_lst |
| **ΔLST Classes** | Classification of ΔLST: high, medium, low delta | delta_lst_class |
| **Urban Heat Exposure Index (UHEI)** | Composite index: LST + (1 – NDVI) + (1 – Albedo) | urban_heat_exposure_index | Composite index: LST + (1 – NDVI) + (1 – Albedo) | urban_heat_exposure_index |

1. **Load AOI (Bologna)** from precise administrative boundary (GeoJSON)
2. **Retrieve satellite datasets**:
   - **Landsat 8/9 SR** for LST, NDVI, and Albedo (summer 2024)
   - **MODIS LST** for day/night surface temperature (summer 2024)
3. **Calculate indices**:
   - **LST (Land Surface Temperature)**: from ST_B10 (Landsat)
   - **Z-score of LST**: statistical anomaly per pixel
   - **NDVI**: vegetation index from SR_B4 and SR_B5
   - **Albedo**: surface reflectance from SR_B2, B4, B5, B6, B7 using Liang (2001) coefficients [1]
   - **Heat-Vegetation Index**: LST norm – NDVI norm
   - **Heat Retention Index**: LST norm – Albedo norm
   - **MODIS ΔLST (Day – Night)** and its classification
   - **Urban Heat Exposure Index (UHEI)**: composite score LST + (1 – NDVI) + (1 – Albedo)
4. **Classify and export outputs**:
   - Classify Z-score, Albedo, ΔLST
   - Export all outputs as raster and GeoJSON (dissolved)

📚 **Reference**

[1] Liang, S. (2001). *Narrowband to broadband conversions of land surface albedo I: Algorithms*. Remote Sensing of Environment, 76(2), 213–238.  
https://doi.org/10.1016/S0034-4257(00)00205-4

In [135]:
import ee
import os
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio
from rasterio import features
from shapely.geometry import shape
from shapely.ops import unary_union
import json
from shapely.geometry import MultiPolygon
from rasterio.features import shapes
from shapely.geometry import MultiPolygon
from shapely.ops import unary_union, polygonize
import maplibre
from ipyleaflet import Map, GeoJSON, LayersControl, WidgetControl
from ipywidgets import HTML, FloatSlider, VBox, Output
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import copy
from sklearn.preprocessing import KBinsDiscretizer
from rasterio.features import rasterize



In [136]:
DATADIR="data"
os.makedirs(DATADIR, exist_ok=True)
BOUNDARY = "bologna_borders.geojson"
YEAR= "2024"
GEPROJECT= 'ndvi-423516'

In [137]:
def kelvin_to_celsius(img):
    return img.select("ST_B10").multiply(0.00341802).add(149.0).subtract(273.15).rename("LST")

In [138]:
def compute_ndvi(img):
    sr = img.select(["SR_B5", "SR_B4"]).multiply(0.0000275).add(-0.2)
    ndvi = sr.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")
    return img.addBands(ndvi)

In [139]:
def compute_albedo(img):
    sr = img.multiply(0.0000275).add(-0.2)
    b2 = sr.select("SR_B2")
    b4 = sr.select("SR_B4")
    b5 = sr.select("SR_B5")
    b6 = sr.select("SR_B6")
    b7 = sr.select("SR_B7")
    albedo = (b2.multiply(0.356)
                .add(b4.multiply(0.130))
                .add(b5.multiply(0.373))
                .add(b6.multiply(0.085))
                .add(b7.multiply(0.072))
                .subtract(0.0018))
    return img.addBands(albedo.rename("Albedo"))

In [140]:
def raster_to_geojson(raster_path, geojson_path,ingdf_boundary,dissolve=False):
    with rasterio.open(raster_path) as src:
        image = src.read(1)
        mask = image != src.nodata
        results = (
            {'properties': {'value': str(v)}, 'geometry': shape(s)}
            for s, v in shapes(image, mask=mask, transform=src.transform)
        )
        gdf = gpd.GeoDataFrame.from_features(list(results), crs=src.crs)
        if dissolve:
            gdf = gdf.dissolve(by='value').reset_index()
       
        if ingdf_boundary is not None:
            gdf = gpd.overlay(gdf, ingdf_boundary, how='intersection')
        gdf = gdf[['value', 'geometry']]
        gdf.to_file(geojson_path, driver='GeoJSON')

In [141]:
def geojson_to_raster(gdf, raster_path,field='value',crs="None",resolution=0.0001):
    # defalt resolution in degrees (adjust as needed)
    if crs != "None":
        gdf = gdf.to_crs(crs)
    # Calculate bounds and resolution
    bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]

    width = int((bounds[2] - bounds[0]) / resolution)
    height = int((bounds[3] - bounds[1]) / resolution)
    transform = rasterio.transform.from_origin(bounds[0], bounds[3], resolution, resolution)

    # Rasterize using the field in 'field'
    shapes = ((geom, int(value)) for geom, value in zip(gdf.geometry, gdf[field]))

    with rasterio.open(
        raster_path,
        "w",
        driver="GTiff",
        height=height,
        width=width,
        count=1,
        dtype="int32",
        crs=gdf.crs,
        transform=transform,
        nodata=0
    ) as dst:
        raster = rasterize(
            shapes=shapes,
            out_shape=(height, width),
            fill=0,
            transform=transform,
            dtype="int32"
        )
        dst.write(raster, 1)

## Load AOI 
Load precise municipal boundary of Bologna from local GeoJSON<br/>
Required for spatial clipping of all datasets


In [142]:
collectdata = False
gdf_boundary = gpd.read_file(BOUNDARY).to_crs("EPSG:4326")
boundary_geom = None

In [143]:
def initialize_gee(gdf_boundary):
    if not ee.data._credentials:
        try:
            ee.Authenticate()
            ee.Initialize(project=GEPROJECT)
            fc = geemap.geopandas_to_ee(gdf_boundary)
            boundary_geom = fc.geometry()
        except Exception as e:
            print(f"I can't initialize GEE: {e}")
            return None

# Data Preparation
## Extract data from Google Earth Engine



## Retrieve MODIS LST (Day/Night) from MOD11A1 – Summer 2024 
MODIS Terra (MOD11A1) provides daily 1 km LST for day and night<br/>
We'll calculate the difference between day and night to assess heat retention
## Classify ΔLST Day-Night 
Δ LST Day-Night (deciles)
This indicator measures the difference in surface temperature between day and night.
It is divided into 10 classes (deciles), from lowest (class 1) to highest (class 10) temperature drop.
- Class 1: Very low Δ → the area retains heat during the night (critical for thermal stress).
- Class 10: Very high Δ → the area cools down effectively at night.
<br/>
This scale helps identify urban zones at risk due to poor nighttime heat dissipation.

In [144]:
if not os.path.exists(DATADIR + os.sep + 'delta_lst_class.tif'):
    initialize_gee(gdf_boundary)
    modis = ee.ImageCollection("MODIS/061/MOD11A1") \
    .filterBounds(boundary_geom) \
    .filterDate(YEAR+"-06-01", YEAR+"-08-31")
    lst_day_modis = modis.select("LST_Day_1km").mean().multiply(0.02).subtract(273.15).rename("LST_Day")
    lst_night_modis = modis.select("LST_Night_1km").mean().multiply(0.02).subtract(273.15).rename("LST_Night")
    delta_lst = lst_day_modis.subtract(lst_night_modis).rename("DeltaLST")
    geemap.ee_export_image(delta_lst.clip(boundary_geom), filename=DATADIR + os.sep + 'delta_lst.tif', scale=1000, region=boundary_geom)
    raster_to_geojson(DATADIR + os.sep + 'delta_lst.tif', DATADIR + os.sep + 'delta_lst.geojson',ingdf_boundary=gdf_boundary, dissolve=False)
    delta_lst = gpd.read_file(DATADIR + os.sep + 'delta_lst.geojson')
    delta_lst["class"] = pd.qcut(pd.to_numeric(delta_lst["value"], errors="coerce"), 10, labels=False)
    delta_lst.to_file(DATADIR + os.sep + 'delta_lst_class.geojson', driver='GeoJSON')
    delta_lst['class'] = delta_lst['class']+1
    geojson_to_raster(delta_lst, DATADIR + os.sep + 'delta_lst_class.tif', field='class')
    geemap.ee_export_image(lst_day_modis.clip(boundary_geom), filename=DATADIR + os.sep + 'lst_day_modis.tif', scale=1000, region=boundary_geom)
    raster_to_geojson(DATADIR + os.sep + 'lst_day_modis.tif', DATADIR + os.sep + 'lst_day_modis.geojson',ingdf_boundary=gdf_boundary, dissolve=True)
    delta_lst = gpd.read_file(DATADIR + os.sep + 'delta_lst.geojson')
    geemap.ee_export_image(lst_night_modis.clip(boundary_geom), filename=DATADIR + os.sep + 'lst_night_modis.tif', scale=1000, region=boundary_geom)
    raster_to_geojson(DATADIR + os.sep + 'lst_night_modis.tif', DATADIR + os.sep + 'lst_night_modis.geojson',ingdf_boundary=gdf_boundary, dissolve=True)
    delta_lst = gpd.read_file(DATADIR + os.sep + 'delta_lst.geojson')


# Calculate Urban Heat Exposure Index (UHEI): LST + (1-NDVI) + (1-Albedo)
- This composite score highlights areas with high temperature, low vegetation, and low albedo
- Normalization ranges: LST (30–50°C), NDVI (0–0.8), Albedo (0.05–0.35)
- Higher UHEI → higher exposure to urban heat conditions
## Retrieve Landsat 8/9 Collection 2 L2 imagery (summer 2024) 
- Purpose: estimate Land Surface Temperature (LST) from band ST_B10
- LST will be used as baseline to analyze surface thermal variability
### Calculate LST Z-score
- Identify statistical anomalies in LST relative to the city's average
- Z = (pixel - mean) / std_dev
### Classify LST Z-score into 10 classes (5 below, 5 above average) 
This helps in highlighting both cool and hot zones relative to the norm

In [145]:
if not os.path.exists(DATADIR + os.sep + 'lst_mean.tif'):
    initialize_gee(gdf_boundary)
    collection = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
        .filterBounds(boundary_geom)
        .filterDate(YEAR+"-06-01", YEAR+"-08-31")
        .filter(ee.Filter.lt("CLOUD_COVER", 10))
        .map(kelvin_to_celsius))
    lst_mean = collection.mean().clip(boundary_geom)
    mean_dict = lst_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=boundary_geom, scale=100, maxPixels=1e13)
    std_dict = lst_mean.reduceRegion(reducer=ee.Reducer.stdDev(), geometry=boundary_geom, scale=100, maxPixels=1e13)
    mean_val = ee.Number(mean_dict.get("LST"))
    std_val = ee.Number(std_dict.get("LST"))
    z_score = lst_mean.subtract(mean_val).divide(std_val).rename("Z")
    classified = z_score.expression("(z <= -2.0) ? 1 : (z <= -1.5) ? 2 : (z <= -1.0) ? 3 : (z <= -0.5) ? 4 : " +
                                        "(z < 0) ? 5 : (z < 0.5) ? 6 : (z < 1.0) ? 7 : (z < 1.5) ? 8 : (z < 2.0) ? 9 : 10", {"z": z_score}).rename("Z_class")
    geemap.ee_export_image(lst_mean.clip(boundary_geom), filename=DATADIR + os.sep + 'lst_mean.tif', scale=100, region=boundary_geom)
    raster_to_geojson(DATADIR + os.sep + 'lst_mean.tif', DATADIR + os.sep + 'lst_mean.geojson',ingdf_boundary=gdf_boundary,dissolve=True)
    geemap.ee_export_image(z_score.clip(boundary_geom), filename=DATADIR + os.sep + 'z_score.tif', scale=100, region=boundary_geom)
    raster_to_geojson(DATADIR + os.sep + 'z_score.tif', DATADIR + os.sep + 'z_score.geojson',ingdf_boundary=gdf_boundary,dissolve=True)
    geemap.ee_export_image(classified.clip(boundary_geom), filename=DATADIR + os.sep + 'z_score_class.tif', scale=100, region=boundary_geom)
    raster_to_geojson(DATADIR + os.sep + 'z_score_class.tif', DATADIR + os.sep + 'z_score_class.geojson',ingdf_boundary=gdf_boundary,dissolve=True)
    
    


##  Compute NDVI (Normalized Difference Vegetation Index) 
NDVI = (NIR - Red) / (NIR + Red)<br/>
Derived from Surface Reflectance bands B5 (NIR) and B4 (Red)<br/>
NDVI is used to identify vegetated vs built-up areas
### Compute Albedo using Liang (2001) method
Albedo is calculated using a weighted sum of Surface Reflectance bands<br>
Equation adapted for Landsat 8: B2, B4, B5, B6, B7<br>
Result: surface reflectance from 0 (fully absorbent) to 1 (fully reflective)
### Normalize LST, NDVI, and Albedo 
- Normalize LST (30–50°C), NDVI (0–0.8), Albedo (0.05–0.35) to [0, 1] range
- Needed for consistent index construction
### Compute Heat-Vegetation Index 
High value = hot and poorly vegetated → risk of UHI
### Compute Heat Retention Index
High value = hot and low albedo → likely to retain heat during night
### Urban Heat Exposure Index (UHEI)
High value = hot, poorly vegetated, and low reflectivity → highest exposure to urban heat risk


In [146]:
if not os.path.exists(DATADIR + os.sep + 'ndvi_mean.tif'):
    ndvi_collection = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
        .filterBounds(boundary_geom)
        .filterDate(YEAR+"-06-01", YEAR+"-08-31")
        .filter(ee.Filter.lt("CLOUD_COVER", 10))
        .map(compute_ndvi))

    ndvi_mean = ndvi_collection.select("NDVI").mean().clip(boundary_geom)
    albedo_collection = ndvi_collection.map(compute_albedo)
    albedo_mean = albedo_collection.select("Albedo").mean().clip(boundary_geom)
    lst_norm = lst_mean.unitScale(30, 50).rename("LST_norm")
    ndvi_norm = ndvi_mean.unitScale(0, 0.8).rename("NDVI_norm")
    albedo_norm = albedo_mean.unitScale(0.05, 0.35).rename("Albedo_norm")
    
    heat_veg_index = lst_norm.subtract(ndvi_norm).rename("HeatVegIndex")
    heat_retention_index = lst_norm.subtract(albedo_norm).rename("HeatRetentionIndex")
    

    uhe_index = lst_norm.add(ee.Image(1).subtract(ndvi_norm)) \
                      .add(ee.Image(1).subtract(albedo_norm)) \
                      .rename("UrbanHeatExposureIndex")

    geemap.ee_export_image(ndvi_mean.clip(boundary_geom), filename=DATADIR + os.sep + 'ndvi_mean.tif', scale=30, region=boundary_geom)
    raster_to_geojson(DATADIR + os.sep + 'ndvi_mean.tif', DATADIR + os.sep + 'ndvi_mean.geojson',ingdf_boundary=gdf_boundary,dissolve=True)
    geemap.ee_export_image(albedo_mean.clip(boundary_geom), filename=DATADIR + os.sep + 'albedo_mean.tif', scale=30, region=boundary_geom)
    raster_to_geojson(DATADIR + os.sep + 'albedo_mean.tif', DATADIR + os.sep + 'albedo_mean.geojson',ingdf_boundary=gdf_boundary,dissolve=True)
    geemap.ee_export_image(heat_veg_index.clip(boundary_geom), filename=DATADIR + os.sep + 'heat_veg_index.tif', scale=100, region=boundary_geom)
    raster_to_geojson(DATADIR + os.sep + 'heat_veg_index.tif', DATADIR + os.sep + 'heat_veg_index.geojson',ingdf_boundary=gdf_boundary,dissolve=True)
    geemap.ee_export_image(heat_retention_index.clip(boundary_geom), filename=DATADIR + os.sep + 'heat_retention_index.tif', scale=100, region=boundary_geom)
    raster_to_geojson(DATADIR + os.sep + 'heat_retention_index.tif', DATADIR + os.sep + 'heat_retention_index.geojson',ingdf_boundary=gdf_boundary,dissolve=True)
    geemap.ee_export_image(uhe_index.clip(boundary_geom), filename=DATADIR + os.sep + 'urban_heat_exposure_index.tif', scale=100, region=boundary_geom)
    raster_to_geojson(DATADIR + os.sep + 'urban_heat_exposure_index.tif', DATADIR + os.sep + 'urban_heat_exposure_index.geojson',ingdf_boundary=gdf_boundary,dissolve=True)
    geemap.ee_export_image(uhe_index.clip(boundary_geom), filename=DATADIR + os.sep + 'urban_heat_exposure_index.tif', scale=100, region=boundary_geom)
    raster_to_geojson(DATADIR + os.sep + 'urban_heat_exposure_index.tif', DATADIR + os.sep + 'urban_heat_exposure_index.geojson',ingdf_boundary=gdf_boundary,dissolve=True)
    geemap.ee_export_image(heat_retention_index.clip(boundary_geom), filename=DATADIR + os.sep + 'heat_retention_index.tif', scale=100, region=boundary_geom)
    raster_to_geojson(DATADIR + os.sep + 'heat_retention_index.tif', DATADIR + os.sep + 'heat_retention_index.geojson',ingdf_boundary=gdf_boundary,dissolve=True)
